In [5]:
import pandas

parquet_file_path = "/root/autodl-tmp/linjh/med_report_R1/dataset/processed/geometry3k/data/train-00000-of-00001.parquet"
df = pandas.read_parquet(parquet_file_path)
df_dict = df.to_dict("list")
df.head()

,images,problem,answer
0,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,<image>Find x.,3
1,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,<image>If $\overline{B E} \cong \overline{E D}...,120
2,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,<image>Find x.,2 \sqrt { 221 }
3,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,<image>Find $x$ so that $m || n$.,27
4,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,"<image>GRID IN In the figure, the radius of ci...",27


In [6]:
from pprint import pprint

sample_idx = 0
pprint(df_dict["images"][sample_idx])
pprint(df_dict["problem"][sample_idx])
pprint(df_dict["answer"][sample_idx])

array([{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x01\xa4\x08\x02\x00\x00\x00\xc8\x7f\n\xc8\x00\x00\x80\x00IDATx\xda\xb4\xbdY\xcc\xb6\xffY\x96\xfb&\x86\xe5\xb4\xe1\x14\xc1(\x0e(\x0bX\x02U\x0b\x0b\x90U\xa0\x02\x9dV[(\x1d\x05\x96\xd42(\x88\xb0d\x9e-3\x85\x0e\xff\xd2B\xa1\xf3\x80\xa2n\x10#\xc6\r\x12\xb2\xd4\xc4h\x948\xd5\xc4-ct\xc3\x1dc\xc48\xec\xaf\xab\xef\xc1wp|\xd7\xef~\xde\xffW\x94\xdf\xc6\x93\xfb}\xdeg\xb8\x9f{\xb8\x86\xf3:\xaf\xf3\xba\xfb\xac\xcf\xfa\xacg?\xfb\xd9\x9f\xf7y\x9f\xf7\x9c\xc7\xd7\xe7g=\xf7\xb9\xcf}\xde\xf3\x9e\xf7\xfc\xfb\xc5\xc6\x0b\xee\xd7\xff\x9d\xf5\xc2\xfb\xf5\xa2\xfb\xf5\x05_\xf0\x05_\xf8\x85_\xf8\x92G\xeb\x8b\xbe\xe8\x8b^\xfah\xbd\xece/{\xf9\xfdze\xd6\xab^\xf5\xaa?}\xbff\x83?}\xc6\xf5\xc5_\xfc\xc5_\xf2%_\xf2\xa5\xf7\xebK\xb2\xbe8\x8b\x97u\xe3\\_zc\xfd?\xb7\x97_\xda\xcf\x99\xe7y\xe6\x81}p\xb7Y\xf3!\xdd\x01\xdf~k\xb9W~\xc5\xc3\xbfk^\xf9g\xee\xd7\x97}\xd9\x97\xf1\xae\xb5o\x0f\x7f\x91\xef\xfd\xb3\x8f\xd6\xab_\xfd\xea\xd7\xbc\xe65_\xfe\xe5

In [4]:
import json
import pandas as pd
from pathlib import Path

# Convert origin dataset to the format required by the easy-r1
def data_convert(data):
    converted_data = {
        "images": [],
        "problem": [],
        "answer": []
    }
    for item in data:
        converted_data["images"].append([
           Path(image_path).read_bytes() for image_path in item["image_path"]
        ])
        converted_data["problem"].append(
            "<image>" * len(item["image_path"]) + \
            "You are an experienced radiologist skilled in analyzing chest X-rays. Please analyze this chest X-ray and respond in exact format: <think>[Your radiological reasoning, including findings and differential diagnosis]</think><answer>[Your final report, containing clear observations and recommendations, should be a paragraph]</answer>"
        )
        converted_data["answer"].append(item["answer"])
    return converted_data
    
# Convert JSON file to Parquet format
def json_to_parquet(json_file_path, parquet_file_path):
    with open(json_file_path, "r") as f:
        data = json.load(f)
    
    converted_data = data_convert(data)
    
    df = pd.DataFrame(converted_data)
    df.to_parquet(parquet_file_path, index=False)
    pass

In [ ]:
import os
import pandas as pd

json_folder = "/root/autodl-tmp/wh/med_report_R1/assets/disease_samples_test"
json_file_path = "/root/autodl-tmp/wh/med_report_R1/assets/disease_samples_test.json"
parquet_file_path = "/root/autodl-tmp/wh/med_report_R1/assets/disease_samples_test.parquet"
data = []
sample_num = 100
for fn in os.listdir(json_folder):
    if fn.endswith(".json"):
        with open(os.path.join(json_folder, fn), "r") as f:
            json_data = json.load(f)
        if len(data) < sample_num:
            json_data = json_data
        else:
            json_data = json_data[:sample_num]
        for item in json_data:
            item["answer"] = json.dumps({
                "disease": fn.split(".")[0],
                "report": item["report"].replace("\n", " ")
            })
            json.loads(item["answer"])  # Ensure the answer is valid JSON
            data.append(item)
with open(json_file_path, "w") as f:
    json.dump(data, f, indent=4)
json_to_parquet(json_file_path, parquet_file_path)